BCOW算法（Binary Counter On Words）是一种快速计算字符串的特定函数值的算法，可以用于字符串匹配、编辑距离计算等问题。

BCOW算法将输入字符串拆分成多个字长为w的字串，并将每个字串转换成一个二进制数。然后对这些二进制数进行计算，得到一个长度为k的二进制序列。最终的函数值就是这个二进制序列转换成十进制的结果。

具体实现中，BCOW算法使用了两个字典，一个用于存储每个字串对应的二进制数，另一个用于记录每个二进制序列出现的次数。具体步骤如下：

1. 将输入字符串按照字长w进行划分，得到n个字串。

2. 对于每个字串，将其映射成一个唯一的二进制数。这里可以采用哈希函数或者其他编码方式实现。

3. 遍历所有的二进制数，统计每个二进制序列出现的次数。这里可以使用位运算和哈希表实现。

4. 对于每个字串，根据其对应的二进制数，在第三步得到的统计结果中查找对应的次数。

5. 将每个字串对应的次数作为二进制位，组成一个长度为k的二进制序列。最终的函数值就是这个二进制序列转换成十进制的结果。

BCOW算法的时间复杂度为O(nw)，其中n为字串个数，w为字长。相比于其他字符串计算算法，如KMP算法、AC自动机等，BCOW算法具有更快的计算速度和更小的空间消耗。

CBOW（Continuous Bag-of-Words）算法是一种将词语转换为数字向量的算法，也称为词嵌入（Word Embedding）技术。

具体来说，CBOW算法通过输入一个词语周围的上下文词语，来预测这个词语本身。在这个过程中，CBOW算法使用一个神经网络模型，并将每个词语表示为一个固定长度的向量，以便于计算和处理。

CBOW算法的输出结果是一个数字向量，它代表了输入词语在特定语境下的含义和语义信息。这个向量可以被用于许多自然语言处理任务，例如文本分类、情感分析、语音识别等。

In [7]:
import itertools
#@save
from torch import nn
import torch
from torch.nn.functional import cross_entropy,softmax
import numpy as np
import matplotlib.pyplot as plt

corpus = [
    # numbers
    "5 2 4 8 6 2 3 6 4",
    "4 8 5 6 9 5 5 6",
    "1 1 5 2 3 3 8",
    "3 6 9 6 8 7 4 6 3",
    "8 9 9 6 1 4 3 4",
    "1 0 2 0 2 1 3 3 3 3 3",
    "9 3 3 0 1 4 7 8",
    "9 9 8 5 6 7 1 2 3 0 1 0",

    # alphabets, expecting that 9 is close to letters
    "a t g q e h 9 u f",
    "e q y u o i p s",
    "q o 9 p l k j o k k o p",
    "h g y i u t t a e q",
    "i k d q r e 9 e a d",
    "o p d g 9 s a f g a",
    "i u y g h k l a s w",
    "o l u y a o g f s",
    "o p i u y g d a s j d l",
    "u k i l o 9 l j s",
    "y g i s h k j l f r f",
    "i o h n 9 9 d 9 f a 9",
]

class CBOW(nn.Module):
    def __init__(self,v_dim,emb_dim):
        super(CBOW, self).__init__()
        self.v_dim=v_dim        #词汇表中单词的数量
        self.embeddings = nn.Embedding(v_dim,emb_dim)   #嵌入层
        #它创建了一个PyTorch嵌入层（Embedding Layer），该层将用于将输入序列中的单词转换为嵌入向量。具体来说，嵌入层将创建一个大小为(v_dim, emb_dim)的权重矩阵，并使用这个矩阵将每个输入单词映射到其对应的嵌入向量。
        self.embeddings.weight.data.normal_(0,0.1)
        # 对一个神经网络模型的参数进行初始化。具体来说，该代码针对模型中的 self.embeddings 权重矩阵进行初始化。其中 normal_ 表示使用正态分布随机初始化权重矩阵，括号内的两个参数 0 和 0.1 分别表示该正态分布的均值和标准差。因此，这行代码的意思是将 self.embeddings 权重矩阵的所有元素都随机初始化为均值为 0、标准差为 0.1 的正态分布中的随机数。

        # self.opt=torch.optim.Adam(0.01)
        self.hidden_out = nn.Linear(emb_dim,v_dim)  #线性层，将嵌入向量映射到输出向量。即它将大小为emb_dim的输入向量映射到大小为v_dim的输出向量。
        self.opt = torch.optim.SGD(self.parameters(),momentum=0.9,lr=0.01)

    def forward(self,x,training=None, mask=None):
        # x.shape = [n,skip_window*2]
        o = self.embeddings(x)  # [n, skip_window*2, emb_dim]
        #嵌入层，将输入的特征向量x映射到低维度的向量空间中，从而压缩特征
        o = torch.mean(o,dim=1) # [n, emb_dim]
        #在第二个维度上进行平均，即对skip_window*2维度进行平均。将o从[n, skip_window*2, emb_dim]降维到[n, emb_dim]，从而获得更为紧凑的特征向量
        return o

    def loss(self, x, y, training=None):
        embedded = self(x,training) #将输入数据x通过调用类实例self的__call__方法得到嵌入向量embedded
        #__call__方法定义了当对象被“调用”时要执行的操作。在这个函数中，__call__方法会将输入传递到模型的前馈传递阶段，即将输入数据转换为嵌入向量。
        pred= self.hidden_out(embedded)
        return cross_entropy(pred,y)    #交叉熵损失函数

    def step(self,x,y):
        self.opt.zero_grad()    #将self对象的梯度（gradient）清零（zero_grad），以便进行下一次反向传播时不受上一次计算影响
        loss = self.loss(x,y,True)#需要对loss进行求解。将计算得到的loss赋值给变量loss
        loss.backward()
        self.opt.step() #使用优化器（Optimizer）更新模型参数（model parameters），以使得loss最小化。
        return loss.detach().numpy()    #返回loss的数值（value），采用numpy()方法将其转换为numpy数组类型，并使用detach()方法从计算图中分离出来。

def train(model,data):
    if torch.cuda.is_available():
        print("GPU train avaliable")
        device =torch.device("cuda")
        model = model.cuda()
    else:
        device = torch.device("cpu")
        model = model.cpu()
    for t in range(2500):
        bx,by = data.sample(16)
        bx,by = torch.from_numpy(bx).to(device), torch.from_numpy(by).to(device)
        loss = model.step(bx,by)
        if t%200 == 0:
            print(f"step: {t}  |  loss: {loss}")


class Dataset:
    def __init__(self,x,y,v2i,i2v):
        self.x,self.y = x,y
        self.v2i, self.i2v = v2i,i2v
        self.vocab = v2i.keys()

    def sample(self,n):
        b_idx = np.random.randint(0,len(self.x),n)
        bx,by = self.x[b_idx],self.y[b_idx]
        return bx,by
    @property
    def num_word(self):
        return len(self.v2i)

def show_w2v_word_embedding(model,data,path):
    word_emb = model.embeddings.weight.data.numpy()
    for i in range(data.num_word):
        c = "blue"
        try:
            int(data.i2v[i])
        except:
            c = "red"

        plt.text(word_emb[i,0],word_emb[i,1], s= data.i2v[i], color=c,weight = "bold")

    plt.xlim(word_emb[:,0].min() - 0.5, word_emb[:,0].max()+0.5)
    plt.ylim(word_emb[:,1].min() - 0.5, word_emb[:,1].max()+0.5)
    plt.xticks(())
    plt.yticks(())
    plt.xlabel("embedding dim1")
    plt.ylabel("embedding dim2")
    plt.savefig(path,dpi=300,format="png")
    plt.show()

def process_w2v_data(corpus,skip_window=2,method = "skip_gram"):
    all_words = [sentence.split(" ") for sentence in corpus]
    # groups all the iterables together and produces a single iterable as output
    all_words = np.array(list(itertools.chain(*all_words)))
    #itertools.chain() 函数可以将多个序列（iterables）连接起来，返回一个新的迭代器对象，其中包含所有序列中的元素。参数 all_words 表示将传入的 all_words 序列作为可迭代对象传入函数， 操作符用于解包 all_words 序列中的所有元素。
    vocab,v_count = np.unique(all_words,return_counts=True) #去重
    """这段代码是在使用NumPy库中的unique函数，对于一个给定的一维数组 all_words进行操作。
    - np.unique: 这是NumPy库中的去重函数，它将返回具有唯一值的数组。
    - all_words: 这是输入的一维数组，其中包含了多个词汇。
    - return_counts=True: 此参数用于告诉 NumPy 按顺序返回给定数据集的唯一元素及其出现次数。
    因此，该代码的作用是从 all_words 数组中找出所有不同的单词，并计算每个单词在数组中出现的次数。它会返回两个数组：
    - 第一个数组是所有不同的单词列表（按字母顺序排序）。
    - 第二个数组是与第一个数组中的单词列表相对应的每个单词出现的频率或次数。"""
    vocab = vocab[np.argsort(v_count)[::-1]]    #按出现次数排序

    print("All vocabularies are sorted by frequency in decresing oreder")
    v2i = {v:i for i,v in enumerate(vocab)} #由向量找索引
    i2v = {i:v for v,i in v2i.items()}      #由索引找向量

    pairs = []
    js = [i for i in range(-skip_window,skip_window+1) if i!=0] #js是滑动窗口内的几个数字组成的列表

    for c in corpus:
        words = c.split(" ")
        w_idx = [v2i[w] for w in words] #对words中的每一个单词，从词表中查找对应的索引
        if method == "skip_gram":   #
            for i in range(len(w_idx)): #第i个滑动窗口
                for j in js:            #第i个滑动窗口中的第j个位置
                    if i+j<0 or i+j>= len(w_idx):
                        continue
                    pairs.append((w_idx[i],w_idx[i+j])) #将每个滑动窗口的单词位置两两配对关联上
            #如果是skip-gram算法，就要生成单词对。对于每个单词，以它为中心，在窗口大小范围内，枚举所有可能的上下文词（即距离它的距离不超过窗口大小的其他单词），并将此单词和其每个上下文词组成一组单词对，存入pairs数组中。
            #skip-gram算法通过每个单词和其上下文词之间的关系来训练模型，从而预测这些上下文词可能对应的中心词。
        elif method.lower() == "cbow":
            for i in range(skip_window,len(w_idx)-skip_window):
                context = []
                for j in js:
                    context.append(w_idx[i+j])  #直接存储滑动窗口中的文本
                pairs.append(context+[w_idx[i]])
            #如果是cbow算法，则需要生成输入和输出词语。对于每个单词，以它为中心，在窗口大小范围内，找到所有上下文词，并将它们作为输入，此单词作为输出，存入pairs数组中。
            #cbow算法则使用上下文词来预测中心词。
        else:
            raise ValueError

    pairs = np.array(pairs)
    print("5 expample pairs:\n",pairs[:5])
    if method.lower()=="skip_gram":
        x,y = pairs[:,0],pairs[:,1]
    elif method.lower() == "cbow":
        x,y = pairs[:,:-1],pairs[:,-1]
    else:
        raise ValueError
    return Dataset(x,y,v2i,i2v)

if __name__ == "__main__":
    d = process_w2v_data(corpus,skip_window=2, method="cbow")
    m = CBOW(d.num_word, 2)
    train(m,d)

    show_w2v_word_embedding(m,d,"./visual/results/cbow.png")

All vocabularies are sorted by frequency in decresing oreder
5 expample pairs:
 [[16 14 12  3  9]
 [14  9  3 14 12]
 [ 9 12 14  1  3]
 [12  3  1  3 14]
 [ 3 14  3  9  1]]
GPU train avaliable


RuntimeError: "nll_loss_forward_reduce_cuda_kernel_2d_index" not implemented for 'Int'